# 2. Redocking

Before we can start our virtual screening campaign, we need to make sure that our docking setup can adequately reproduce the pose of protein/co-ligand complex in the crystal structure. In practice, this would mean docking the the co-ligand, starting from its structure in the PDB, and checking to see if the RMSD between the docked and the crystal structure is less that 2 Å.

For this, we will need to prepare the co-ligand. We add hydrogens using Openbabel (`obabel`) and use the Meeko helper tool to prepare the ligand for docking with Autodock Vina. The binaries are listed below:

In [1]:
## define all binaries
mk_prepapre_ligand = 'mk_prepare_ligand.py'
vina = 'vina'
obabel = 'obabel'

Protonate the ligand at physiological pH, and prepare it:

In [2]:
ligand_file = '0G6_H'
!obabel /Users/vbf/Work/projects/terraq/1ppb/vina/2_redock/1_ligand/1ppb_C_0G6.sdf -O{ligand_file}.sdf -ph 7.4

1 molecule converted


In [3]:
!{mk_prepapre_ligand} -i {ligand_file}.sdf -o {ligand_file}.pdbqt

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


The main parameter we can play with when perfoming docking in Autodock Vina is the exhaustiveness. A higher exhaustiveness value means more search runs are performed, leading to a more thorough exploration of the conformational space and a greater probability of finding the lowest-energy binding pose. But, would of course require more computational effort. Conversely, a lower value is faster but may miss the optimal solution. We perform docking at various exhaustiveness values to check for convergence.

In [4]:
exh_vals = [8, 32, 64, 128]

In [5]:
for i in exh_vals:
    cmd = (f"{vina} --ligand {ligand_file}.pdbqt  --receptor 1ppb_receptor.pdbqt "
       f"--config 1ppb_receptor.box.txt --exhaustiveness={i} --out 2_redock/2_docking/docked{i}.pdb "
        f"--num_modes 9 | tee 2_redock/2_docking/docking{i}.log")
    print(f"Running with exhaustiveness {i}")
    print(cmd)
    !{cmd}

Running with exhaustiveness 8
/Users/vbf/Work/bin/vina_1.2.7_mac_aarch64 --ligand 0G6_H.pdbqt  --receptor 1ppb_receptor.pdbqt --config 1ppb_receptor.box.txt --exhaustiveness=8 --out docked8.pdb --num_modes 9 | tee docking8.log
AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp.

After performing docking, we can check the RMSD and docking scores for convergence.

In [6]:
file_list = [f"2_redock/2_docking/docked{i}.pdb" for i in exh_vals]

Below defines some helper functions to extract the coordinates of the first (best ranked) molecule and score from the docking results.

In [7]:
import re
from rdkit import Chem
from rdkit.Chem import AllChem

def get_first_mol_from_pdbqt(file):
    mol_block = ""
    score = None
    with open(file, 'r') as f:
        for i, line in enumerate(f):
            if i == 1:
                finds = re.findall(r"REMARK VINA RESULT:\s+([0-9\-.]+)", line)
                if finds != []:
                    score = float(finds[0])

            if line.startswith("ATOM"):
                line_ = line.strip()
                mol_block += line_[:len(line_)-2]+'\n'

            if line.strip() == 'ENDMDL':
                break

    return Chem.MolFromPDBBlock(mol_block, removeHs=True), score

mol_original, _ = get_first_mol_from_pdbqt("2_redock/1_ligand/0G6_H.pdbqt")

def get_scores(file):
    mol,score = get_first_mol_from_pdbqt(file)
    return AllChem.GetBestRMS(mol_original, mol), score

In [8]:
dock_scores = []
rmsd_scores = []

In [9]:
for file in file_list:
    i,j = get_scores(file)
    rmsd_scores.append(i)
    dock_scores.append(j)

import pandas as pd

pd.DataFrame({"exhaustiveness": exh_vals, "RMSD": rmsd_scores, "Docking Score": dock_scores})

,exhaustiveness,RMSD,Docking Score
0,8,0.697987,-8.499
1,32,0.689169,-8.529
2,64,0.707369,-8.485
3,128,0.680418,-8.489


We can see above that the RMSD is within the 2 Å for all values of exhaustiveness. Furthermore, there is not much various among the scores as the exhaustiveness changes. Thus, I deem an exhaustiveness value of 8 as sufficient for accurate docking in this system. This setup is used for the next step of virtual screening.